In [2]:
import pickle
import os

data_folder = './dialogsum/DialogSum_Data/'
train_path = os.path.join(data_folder, 'train_dialogsum_sentence_transformer.pkl')
dev_path = os.path.join(data_folder, 'val_dialogsum_sentence_transformer.pkl')
test_path = os.path.join(data_folder, 'test_dialogsum_sentence_transformer.pkl')


with open(train_path, 'rb') as f:
    train = pickle.load(f)
with open(dev_path, 'rb') as f:
    val = pickle.load(f)
with open(test_path, 'rb') as f:
    test = pickle.load(f)

In [4]:
import json
test_json_path = os.path.join(data_folder, 'test_dialogsum.jsonl')

def read_json(file):
    json_arr = []
    lines = file.readlines()
    for line in lines:
        jsn = json.loads(line)
        json_arr.append(jsn)
    return json_arr

with open(test_json_path, encoding = 'utf8') as json_file:
    #data = json.load(json_file)
    data = read_json(json_file)
    
conversations = []
summaries = []
for i in range(0, len(data)):
    if len(data[i]['dialogue'].split('\r\n')) > 1:
        sentences = data[i]['dialogue'].split('\r\n')
    else:
        sentences = data[i]['dialogue'].split('\n')

    conversations.append(sentences)
    summaries.append(data[i]['summary1'].strip('\n').replace('\r\nt', ' '))

In [3]:
import numpy as np
length = []
for i in range(0, len(train)):
    length.append(len(train[i]))
for i in range(0, len(val)):
    length.append(len(val[i]))
for i in range(0, len(test)):
    length.append(len(test[i]))
X = []
for i in range(0, len(train)):
    for j in range(0, len(train[i])):
        X.append(np.array(train[i][j]))
for i in range(0, len(val)):
    for j in range(0,len(val[i])):
        X.append(np.array(val[i][j]))
for i in range(0, len(test)):
    for j in range(0, len(test[i])):
        X.append(np.array(test[i][j]))

In [1]:
from hmmlearn import hmm

ModuleNotFoundError: No module named 'hmmlearn'

In [6]:
remodel = hmm.GaussianHMM(n_components=4, n_iter = 50, covariance_type = 'diag', verbose = True, init_params="cm", params="cmts")
remodel.startprob_ = np.array([1, 0.0, 0.0, 0.0])
remodel.transmat_ = np.array([
                 [0.33, 0.34, 0.33, 0],
                 [0.0, 0.33, 0.34, 0.33],
                 [0.0, 0.0, 0.5, 0.5],
                 [0.0, 0.0, 0.0, 1.0]]
                            )

In [7]:
remodel.startprob_

array([1., 0., 0., 0.])

In [8]:
remodel.transmat_

array([[0.33, 0.34, 0.33, 0.  ],
       [0.  , 0.33, 0.34, 0.33],
       [0.  , 0.  , 0.5 , 0.5 ],
       [0.  , 0.  , 0.  , 1.  ]])

In [9]:
remodel.fit(X, length)

         1   -67062215.8653             +nan
         2   -64879878.7185    +2182337.1468
         3   -64642016.4478     +237862.2707
         4   -64399749.0050     +242267.4428
         5   -64176896.5768     +222852.4281
         6   -63993821.6084     +183074.9685
         7   -63851722.1101     +142099.4982
         8   -63759133.2361      +92588.8740
         9   -63688370.8798      +70762.3564
        10   -63633542.0864      +54828.7933
        11   -63596281.9698      +37260.1166
        12   -63572049.7863      +24232.1835
        13   -63556781.4820      +15268.3043
        14   -63545279.6087      +11501.8734
        15   -63536326.7679       +8952.8408
        16   -63529691.0704       +6635.6975
        17   -63525005.9705       +4685.1000
        18   -63521573.5301       +3432.4404
        19   -63519148.4602       +2425.0699
        20   -63516892.4305       +2256.0297
        21   -63514825.2098       +2067.2207
        22   -63512992.2279       +1832.9818
        23

GaussianHMM(init_params='cm', n_components=4, n_iter=50, params='cmts',
            verbose=True)

In [10]:
remodel.transmat_

array([[0.78158618, 0.10433682, 0.114077  , 0.        ],
       [0.        , 0.18700873, 0.77957176, 0.0334195 ],
       [0.        , 0.        , 0.9131403 , 0.0868597 ],
       [0.        , 0.        , 0.        , 1.        ]])

In [11]:
remodel.startprob_

array([1., 0., 0., 0.])

In [12]:
test_path

'./dialogsum/DialogSum_Data/test_dialogsum_sentence_transformer.pkl'

In [13]:
def encode_convs(profix):
    sent_label = []
    with open(profix + '_sentence_transformer.pkl', 'rb') as f:
        data = pickle.load(f)
    for i in range(0, len(data)):
        labels = remodel.decode(np.array(data[i]))[1]
        sent_label.append(labels)
    
    with open(profix + '_sent_trans_cons_label_2.pkl', 'wb') as f:
        pickle.dump(sent_label, f)
    return sent_label

In [14]:
l = encode_convs('./dialogsum/DialogSum_Data/train_dialogsum')

In [15]:
l = encode_convs('./dialogsum/DialogSum_Data/val_dialogsum')

In [16]:
l = encode_convs('./dialogsum/DialogSum_Data/test_dialogsum')

In [1]:
import pickle
import os
import json
import numpy as np

profix = './dialogsum/DialogSum_Data/val_dialogsum'
sent_label = []
with open(profix + '_sentence_transformer.pkl', 'rb') as f:
    data = pickle.load(f)
for i in range(0, len(data)):
    labels = remodel.decode(np.array(data[i]))[1]
    sent_label.append(labels)

#with open(profix + '_sent_trans_cons_label_2.pkl', 'wb') as f:
#    pickle.dump(sent_label, f)

NameError: name 'remodel' is not defined